In [12]:
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, MatchValue

In [13]:
client = QdrantClient(url="http://localhost:6333")

In [14]:
collection_name = f"test_collection_{uuid.uuid4()}"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

collection_name

'test_collection_2a81a3a1-4e9b-4efc-8852-73a0ce494cd0'

In [15]:
operation_info = client.upsert(
    collection_name=collection_name,
    wait=True,
    points=[
        PointStruct(
            id=1,
            vector=[0.05, 0.61, 0.76, 0.74],
            payload={"city": "Berlin"}
        ),
        PointStruct(
            id=2,
            vector=[0.19, 0.81, 0.75, 0.11],
            payload={"city": "London"}
        ),
        PointStruct(
            id=3,
            vector=[0.36, 0.55, 0.47, 0.94],
            payload={"city": "Moscow"}
        ),
        PointStruct(
            id=4,
            vector=[0.18, 0.01, 0.85, 0.80],
            payload={"city": "New York"}
        ),
        PointStruct(
            id=5,
            vector=[0.24, 0.18, 0.22, 0.44],
            payload={"city": "Beijing"}
        ),
        PointStruct(
            id=6,
            vector=[0.35, 0.08, 0.11, 0.44],
            payload={"city": "Mumbai"}
        ),
    ],
)

operation_info

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
# Let’s ask a basic question - Which of our stored vectors are most similar to the query vector [0.2, 0.1, 0.9, 0.7]?

search_result = client.query_points(
    collection_name=collection_name,
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

search_result

[ScoredPoint(id=4, version=0, score=1.362, payload=None, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=1.273, payload=None, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]

In [17]:
search_result = client.query_points(
    collection_name=collection_name,
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=True,
    limit=3
).points

search_result

[ScoredPoint(id=4, version=0, score=1.362, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=1.273, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=1.208, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None)]

In [18]:
search_result = client.query_points(
    collection_name=collection_name,
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
